## Fase 1: Importar las dependencias

In [1]:
import numpy as np
import math
import re #libreria expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup # trabajar texto codificado en varios formatos(metadatos, XML)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers #de la sublibrería keras saco las capas (layers) 
import tensorflow_datasets as tfds #dentro de tfds hay una herramienta muy útil llamada tokenizador.

## Fase 2: Pre Procesado de Datos

In [19]:
cols = ["sentiment", "id", "date", "query", "user", "text"] #no tiene cabecera el csv

train_data = pd.read_csv(
    "./data/train.csv",
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1"
)

test_data = pd.read_csv(
    "./data/test.csv",
    header = None,
    names = cols,
    engine = "python", #engine : motor de carga python
    encoding = "latin1" #codificación estándar idiomas europeos/americanos (derivados del latin).
)                      

In [20]:
train_data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [21]:
test_data.tail(5) 

,sentiment,id,date,query,user,text
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."
497,0,14076,Sun Jun 14 21:36:17 UTC 2009,iran,captain_pete,Reading the tweets coming out of Iran... The w...


In [22]:
data = train_data

## Pre Procesado

### Limpieza

In [23]:
data.drop(['id','date','query','user'], 
                 axis = 1, #eliminar solo las columnas // axis = 0 (elimina filas)
                 inplace = True) #quiero la versión de data sin las columnas eliminadas.

In [24]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [25]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet,"lxml").get_text()
    #eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #cualquier cosa que empiece por @ y seguido de un número o letra 
    #(usuariotwitter) en cualquier orden. Lo reemplazará por un espacio en blanco ' '.
    
    #eliminamos los links de las URLs
    tweet =re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet) # ?s: el caracter s es opcional. 
                                                         #./ : incluido las barras (en las URLs las hay)
                                                         # Lo reemplaza por un espacio en blanco.
            
    # nos quedamos solo con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']",' ', tweet)          #eliminamos todos los caracteres a excepción (^):
                                                         # a-z, A-Z, ., !, ? '. Lo demás se reemplaza por espacio blanco.
    
    #eliminamos espacios en blanco adicionales que he creado
    tweet = re.sub(r" +", ' ',tweet)                     # busco al menos dos espacios en blanco seguidos uno detrás del otro
                                                         # y los reemplazo por uno solo.
    return tweet

In [28]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

C:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [45]:
#data_clean

In [27]:
set(data.sentiment) #conjunto de valores diferentes que hay en la columna
#el 0 es negativo y el 4 es positivo.

{0, 4}

In [30]:
data_labels = data.sentiment.values #extraigo los valores de nuestro data
data_labels[data_labels == 4] = 1 #filtro las filas donde data_labels = 4 y lo cambio por 1

In [31]:
set(data.sentiment)

{0, 1}

In [32]:
# data_clean  (corpus: lista de todo el texto que se quiere analizar).

### Tokenización.

Ahora nuestro data es una lista se caracteres conformada por palabras en inglés. Lo que quiero hacer ahora es obtener una lista de números y cada uno de los número representará una palabra diferente. Existe una herramienta en TensorFlow que nos permite hacerlo automáticamente. Vamos a construir para cada frase la lista de palabras que contiene. Transformaremos la frase de palabras entendibles por un humano a una lista de número donde cada número referencia a una palabra única.

In [33]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus( #construir tokenizador a partir de un corpus
    data_clean, target_vocab_size = 2**16 #lista de strings y tamaño del vocabulario 2^16.
)
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean] #para cada una de las frase del dataset limpio, le pido que me la codifique
# me devolverá una lista de números donde cada uno de los número corresponde a una palabra diferente.

In [34]:
#data_inputs

### Padding
El entrenamiento en inteligencia artificial muchas veces se hace por bloques que queremos que tengan la misma longitud. Estos bloques permiten hacer un machine learning un aprendizaje que en vez de suministrarle frase a frase se le otorga un conjunto de elementos. Añade 0 al final de cada una de las frases ya que el tokenizador no usa el 0 (no significada nada) y así quedan de la misma longitud.

In [52]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value = 0,
                                                            padding = "post",
                                                            maxlen = MAX_LEN)

In [54]:
data_inputs

array([[65316,  1570,   113, ...,     0,     0,     0],
       [   11,  1090,    23, ...,     0,     0,     0],
       [65316,     3, 41563, ...,     0,     0,     0],
       ...,
       [  927,    12,   229, ...,     0,     0,     0],
       [  366,   337,  1309, ...,     0,     0,     0],
       [  181, 51236,     0, ...,     0,     0,     0]])

### Dividimos en los conjuntos de entrenamiento y testing.

In [55]:
test_idx = np.random.randint(0,800000, 8000) #los identificadores de test entre 0 y 800000 elijo 8000 filas (negativos) (números aleatorios).
test_idx = np.concatenate((test_idx, test_idx+800000)) # le sumo 800000 me encuentro en la sección de los positivos.

In [56]:
test_idx

array([ 505773,  377058,  428258, ...,  965923,  967503, 1482645])

In [37]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis = 0) #se borra por filas
train_labels = np.delete(data_labels, test_idx)

## Construcción del modelo

DCNN: deep convolutional neural network que hereda de tensorflow.keras.Model 
Vamos a heredad de esa clase que ya habíamos importado al inicio de todo. Lo primero que se hace siempre que heredamos de un modelo keras es definir un método init. Definimos nuestro constructor.

vocab_size: tamaño del vocabulario ¿Cuántas palabras queremos manejar? Todavía no lo sabemos y nos lo va a dar el tokenizador.

emb_dim: ¿A qué espacio vectorial vamos a embeber nuestra información? 128 números. Cada palabra será embebida a un espacio vectorial de dimensión 128.

nb_filters:¿Cuántos filtros vamos a utilizar? Filtros de dos, tres y cuatro palabras. He puesto que habrá 50 de cada uno de ellos.

FNN_units: número de neuronas de la DCNN

bn_classes: diferentes tipos de categorías (positivo o negativo)

dropout_rate: previene overfitting y que algunas neuronas se desactiven mientras otras aprenden. El 10% de las neuronas no transmitirán lo que han aprendido en la fase de entrenamiento.

training = False -> Se utilizará para saber si la red neurona tiene que entrenar y por tanto tiene que llevar a cabo la fase de propagación hacia atrás del error y corregir los pesos de la misma o solo se utiliza para lleva a cabo las predicciones.

In [59]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

## Configuración

In [60]:
VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2#len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

## Entrenamiento

In [61]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [63]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [42]:
checkpoint_path = "/home/lucia/Escritorio/Curso_Udemy_PNL/CKP"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/5
 4753/49503 [=>............................] - ETA: 3:49:28 - loss: 0.4680 - accuracy: 0.7763